In [1]:
import sys
sys.path.append('../Utils')

from utils import FFModel, TransformerModelv3, TransformerModelv4, MixtureModelv0, get_num_parameters
from sklearn import model_selection

import torch.optim as optim

import skorch
import joblib

import os

In [2]:
DATASET_DIR = '../Dataset'

In [3]:
ds_info = joblib.load(os.path.join(DATASET_DIR, 'ds_info.jl'))

columns = ds_info['columns']
numerical_cols = ds_info['numerical_columns']
categorical_cols = ds_info['categorical_columns']
label_col = ds_info['target_column']

In [4]:
def get_all_params(param_grid, model):
    count_params = []
    model_params_list = list(model_selection.ParameterGrid(param_grid))
    for model_params in model_params_list:
        model.set_params(**model_params)
        #model.initialize()
        n_params = get_num_parameters(model.module_)
        count_params.append(n_params)
        
    return count_params    

# MLP

In [5]:
# MLP
param_grid = {
    'lr': [1e-3],
    'optimizer': [optim.RMSprop], 
    'module__hidden_sizes': [[32, 16, 8], [32, 32, 16], [64, 32, 32], [128, 64, 32]],
    'module__dropouts': [[0, 0, 0]],
    'module__n_inputs': [len(columns) - 1],
    'module__n_outputs': [1]
}

get_all_params(param_grid, skorch.NeuralNetBinaryClassifier(
    module=FFModel
))

[1153, 2081, 4129, 12289]

# Transformer v3

In [6]:
n_cat_labels = 96

param_grid = {  
    'lr': [1e-3],
    'optimizer': [optim.RMSprop],
    'module__ninp': [8, 16, 32],
    'module__nhead': [1],
    'module__nhid': [16, 32, 64],
    'module__nlayers': [1],
    'module__dropout': [0.1],
    
    # No searcheable parameters
    'module__n_cat_labels': [n_cat_labels],
    'module__n_num_cols': [len(numerical_cols)],
    'module__n_cat_cols': [len(categorical_cols)],
    'module__n_features': [len(columns) - 1]
    
}

get_all_params(param_grid, skorch.NeuralNetClassifier(
    module=TransformerModelv3
))

[1625, 3745, 9521, 1897, 4273, 10561, 2441, 5329, 12641]

# Transformer v4

In [7]:
param_grid = {  
    'lr': [1e-3],
    'optimizer': [optim.RMSprop],
    'module__ninp': [8, 16, 32],
    'module__nhead': [1],
    'module__nhid': [16, 32, 64],
    'module__nlayers': [1],
    'module__dropout': [0.1],
    
    # No searcheable parameters
    'module__n_features': [len(columns) - 1]
    
}

get_all_params(param_grid, skorch.NeuralNetClassifier(
    module=TransformerModelv4
))

[1049, 2593, 7217, 1321, 3121, 8257, 1865, 4177, 10337]

# Mixture model v0

In [8]:
param_grid = {  
    'lr': [1e-3],
    'optimizer': [optim.RMSprop],
    'module__ninp': [8, 16],
    'module__nhead': [1],
    'module__nhid': [16, 32],
    'module__nmodels': [2, 3, 4],
    'module__dropout': [0.15],
    
    # No searcheable parameters
    'module__nfeatures': [len(columns) - 1],
    'module__nclasses': [1]
}

get_all_params(param_grid, skorch.NeuralNetClassifier(
    module=MixtureModelv0
))

[2528, 2560, 2592, 5456, 5488, 5520, 4400, 4448, 4496, 9120, 9168, 9216]